In [226]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import preprocessing
import numpy as np
import pandas as pd
import re as re
import lxml.html
import lxml.html.clean
from nltk.tokenize import RegexpTokenizer
from string import digits
import nltk
from nltk.corpus import stopwords
import pickle
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
import scipy
from scipy.sparse import hstack
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
nltk.download('words')
nltk.download('stopwords')
# from google.colab import drive
# drive.mount('/content/drive')

token = RegexpTokenizer("[\w']+")
sw = stopwords.words('english')
tokenizer = (nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [260]:
train_data = pd.read_csv("dataset/training/data_train.csv", names=['country', 'sku_id', 'title', 'category_lvl_1', 'category_lvl_2', 'category_lvl_3', 'short_description', 'price', 'product_type'])
train_clarity_labels = pd.read_csv("dataset/training/clarity_train.csv", names=['clarity'])
train_conciseness_labels = pd.read_csv("dataset/training/conciseness_train.csv", names=['conciseness'])

validation_data = pd.read_csv("dataset/validation/data_valid.csv", names=['country', 'sku_id', 'title', 'category_lvl_1', 'category_lvl_2', 'category_lvl_3', 'short_description', 'price', 'product_type'])
validation_clarity_labels = pd.read_csv("dataset/validation/clarity_valid.csv", names=['clarity'])
validation_conciseness_labels = pd.read_csv("dataset/validation/conciseness_valid.csv", names=['conciseness'])


In [186]:
train_data.iloc[:-2]

,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type
0,my,AD674FAASTLXANMY,Adana Gallery Suri Square Hijab – Light Pink,Fashion,Women,Muslim Wear,<ul><li>Material : Non sheer shimmer chiffon</...,49.00,local
1,my,AE068HBAA3RPRDANMY,Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz,Health & Beauty,Bath & Body,Hand & Foot Care,Formulated with oil-free hydrating botanicals/...,128.00,international
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,<ul> <li>150cm mini microphone compatible for ...,25.07,international
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,<ul> <li>ANMYNA Complaint Silky Set (Shampoo 5...,118.00,local
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,<ul> <li>100% Authentic</li> <li>Rrefresh and ...,114.80,international
5,my,AS575ELCMZ4WANMY,Asus TP300LJ-DW004H Transformer Book Flip 4GB ...,Computers & Laptops,Laptops,Traditional Laptops,"<div class=""prod_content""> <div class=""prod_de...",2599.00,local
6,my,AS727ELAA9LLV1ANMY,NG-40C Ring-Shaped 40W 3166lm 5400K Macro Phot...,Cameras,Camera Accessories,Lighting & Studio Equipment,<ul> <li>1. Color Temperature: 5400K</li> <li>...,388.99,international
7,my,BU512HBAA4WUVTANMY,Buytra Exfoliating Peel Foot Mask 1Pair,Health & Beauty,Bath & Body,Hand & Foot Care,<ul> <li>Reviving like a new born baby.</li> <...,10.40,international
8,my,CL787ELAW29LANMY,CLiPtec OCC121 Slim Flat USB 3.0 Extension Cab...,Computers & Laptops,Laptops,Traditional Laptops,"<ul style= ""padding: 0px; margin: 20px 0px 0px...",29.00,local
9,my,CO633HLAABREKOANMY,McDonald's Coke Can Glass Limited Edition 12oz...,Home & Living,Kitchen & Dining,Tableware,<ul> <li>Genuine issued McDonald's Coca Cola m...,25.00,local


In [187]:
#train_clarity_labels


In [188]:
# train_conciseness_labels
train_data.shape

(36283, 9)

In [261]:

# preprocessing --
train_data['category_lvl_3'] = train_data['category_lvl_3'].fillna('NA')
#null values check
def null_value_chk(data):
    for column in data.columns:
        print(column, data[column].isnull().sum())


#html tags removing from description
def remove_tags(string):
    
    doc = lxml.html.fromstring(string)
    cleaner = lxml.html.clean.Cleaner(style=True)
    doc = cleaner.clean_html(doc)
    text = doc.text_content()
    return text

def clear_description(data):
    new_string = re.sub('[:!@#$]','',data )
    return new_string

#data cleaning of null vales of category

def category_data(data):
    for idx in range(0 , len(data)):
        if(pd.isnull(data['category_lvl_3'][idx]) ):
            data.loc[idx, 'category_lvl_3'] = data.loc[idx]['category_lvl_1'] + " " + data.loc[idx]['category_lvl_2']


def clean_data(data):
    
    #train_data['short_description'] = train_data['short_description'].apply(lambda x: 'no description')
    #html tags removal
    train_data['short_description'] = train_data['short_description'].apply(lambda cw : remove_tags(str(cw)))
    train_data['short_description'] = train_data['short_description'].apply(lambda x:clear_description(str(x)))


#removing description for now - will see it later
# train_data['short_description'] = train_data['short_description'].apply(lambda x: 'no description')

clean_data(train_data)
train_data.iloc[2:6]



,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,"150cm mini microphone compatible for iPhone, ...",25.07,international
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,ANMYNA Complaint Silky Set (Shampoo 520ml + C...,118.00,local
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,100% Authentic Rrefresh and brighten skin Ant...,114.80,international
5,my,AS575ELCMZ4WANMY,Asus TP300LJ-DW004H Transformer Book Flip 4GB ...,Computers & Laptops,Laptops,Traditional Laptops,Genuine Windows® 8.1 13.3in 169 HD TOUCH In...,2599.00,local


In [190]:
train_data.to_csv('train_preprocessed.csv',encoding='utf-8', index=False)


In [191]:
train_data_preprocessed = pd.read_csv("train_preprocessed.csv", encoding='utf-8')

In [192]:
train_data_preprocessed


,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type
0,my,AD674FAASTLXANMY,Adana Gallery Suri Square Hijab – Light Pink,Fashion,Women,Muslim Wear,Material Non sheer shimmer chiffonSizes 52 x...,49.00,local
1,my,AE068HBAA3RPRDANMY,Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz,Health & Beauty,Bath & Body,Hand & Foot Care,Formulated with oil-free hydrating botanicals/...,128.00,international
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,"150cm mini microphone compatible for iPhone, ...",25.07,international
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,ANMYNA Complaint Silky Set (Shampoo 520ml + C...,118.00,local
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,100% Authentic Rrefresh and brighten skin Ant...,114.80,international
5,my,AS575ELCMZ4WANMY,Asus TP300LJ-DW004H Transformer Book Flip 4GB ...,Computers & Laptops,Laptops,Traditional Laptops,Genuine Windows® 8.1 13.3in 169 HD TOUCH In...,2599.00,local
6,my,AS727ELAA9LLV1ANMY,NG-40C Ring-Shaped 40W 3166lm 5400K Macro Phot...,Cameras,Camera Accessories,Lighting & Studio Equipment,1. Color Temperature 5400K 2. Ra >90 3. Volta...,388.99,international
7,my,BU512HBAA4WUVTANMY,Buytra Exfoliating Peel Foot Mask 1Pair,Health & Beauty,Bath & Body,Hand & Foot Care,Reviving like a new born baby. AgeApply at an...,10.40,international
8,my,CL787ELAW29LANMY,CLiPtec OCC121 Slim Flat USB 3.0 Extension Cab...,Computers & Laptops,Laptops,Traditional Laptops,Designed and constructed to super-speed USB ...,29.00,local
9,my,CO633HLAABREKOANMY,McDonald's Coke Can Glass Limited Edition 12oz...,Home & Living,Kitchen & Dining,Tableware,Genuine issued McDonald's Coca Cola merchandi...,25.00,local


In [264]:
colors_df = set()
brands_df = set()

def __load__():
    global colors_df
    global brands_df
    
    colors_df = set(line.strip().lower() for line in open('dataset\colors.txt'))
    brands_df = set(line.strip().lower() for line in open('dataset\lazada_brands.txt'))

__load__()

def jaccard_sim(s1, s2):
    inter = float(len(s1&s2))
    union = float(len(s1|s2))
    return inter / union  if union > 0 else -1.0

def has_numeric_value(word):
    return bool(re.search(r'\d', word))

def special_char_count(text):
    special_chars = '''!"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~ '''
    return sum(1 for c in text if c in special_chars)

def price_country(price, country):
    
    if country == 'my':
        price *= 0.32
    elif country != 'sg':
        price *= 0.074
    
    return price

def is_color(color):
    global colors_df
    return color.lower() in colors_df

def is_brand(brand):
    global brands_df
    return brand in brands_df


def generate_surface_features(df):
    
    feat_names = ['title_words_count', 'title_unique_words_count', 'title_duplicate_words_count', 'title_duplicate_words_ratio',\
            'title_avg_word_length','title_sku','title_has_numeric_value','title_desc_sim', 'title_cat1_sim',\
            'title_cat2_sim', 'title_cat3_sim', 'title_special_chars_count', 'price_c','title_has_color', 'title_has_brand']
   
    feats_data = []
    
    for i, row in df.iterrows():
        
        #new features
        feat = [] 
        
        title = row['title'].lower()
        desc = row['short_description']
        sku_id = row['sku_id'].lower()
        price = float(row['price'])
        country = row['country']
        
        title_words = title.split(' ')
        title_words_count = len(title_words)
        feat.append(title_words_count)
            
        desc_words = desc.split(' ')
            
            
        #if there are many duplicate words, the title will be not concise
    
        title_unique_words = [w for w in set(title_words)]
        
        title_unique_words_count = len(title_unique_words)
        feat.append(title_unique_words_count)
        
        title_duplicate_words_count = title_words_count - title_unique_words_count
        feat.append(title_duplicate_words_count)
            
        title_duplicate_words_ratio = title_duplicate_words_count / title_words_count
        feat.append(title_duplicate_words_ratio)
        
        title_avg_word_length = len(title)/title_words_count
        feat.append(title_avg_word_length)
               
        
        # if first two characters matches with sku_id. first word in title usually shows brand
        title_sku = int(title[:2] == sku_id[:2])
        feat.append(title_sku)
            
        #contains some numeric value which may show product model
        title_has_numeric_value = int(any([ has_numeric_value(w) for w in title_words ]))
        feat.append(title_has_numeric_value)
        
        #jaccard similarity score between title and description
        title_desc_sim = jaccard_sim(set(title_words),set(desc_words))
        feat.append(title_desc_sim)
        
        
        
        cat1 = row['category_lvl_1'].lower().split()
        cat2 = row['category_lvl_2'].lower().split()
        cat3 = row['category_lvl_3'].lower().split()
        
        # similarity between title words and category words
        feat.append(len(set(title_words)&set(cat1)))
        feat.append(len(set(title_words)&set(cat2)))
        feat.append(len(set(title_words)&set(cat3)))
        
        
        #special character count
        title_special_chars_count = special_char_count(title)
        feat.append(title_special_chars_count)
        
        #price country
        price_c = price_country(price,country)
        feat.append(price_c) 
        
        #title contains color
        title_has_color = int(any([is_color(w) for w in title_words]))
        feat.append(title_has_color)
        
        #title has brand
        title_has_brand = int(any([is_brand(w) for w in title_words]))
        feat.append(title_has_brand)
        
        feats_data.append(feat)
        
    
    return pd.DataFrame(feats_data, columns=feat_names)
    #return scipy.sparse.csr_matrix(feats_data)
        
train_data_feats = generate_surface_features(train_data)

In [231]:
def generate_title_text_features(train_data,validation_data,analyzer="word", n_grams=(2,6), min_df = 1, max_df = 1.0):
    global title_text_vectorizer
    #title text features ngrams
    countVectorizer = CountVectorizer(analyzer=analyzer, ngram_range=n_grams, min_df=min_df, max_df = max_df, lowercase=True)
    X_train = countVectorizer.fit_transform(train_data['title'])
    X_test = countVectorizer.transform(validation_data['title'])
    #print(countVectorizer.get_feature_names())
    return X_train,X_test
  
X_title_count_tr,X_title_count_val = generate_title_text_features(train_data,validation_data, min_df = 0.005)
# train_data_feats['title_text_count_feat'] = generate_title_text_features(train_data)

In [266]:
train_data_feats.head()

,title_words_count,title_unique_words_count,title_duplicate_words_count,title_duplicate_words_ratio,title_avg_word_length,title_sku,title_has_numeric_value,title_desc_sim,title_cat1_sim,title_cat2_sim,title_cat3_sim,title_special_chars_count,price_c,title_has_color,title_has_brand
0,8,8,0,0.0,5.500000,1,0,0.000000,0,0,0,7,15.6800,1,1
1,7,7,0,0.0,7.000000,0,1,0.000000,0,0,0,8,40.9600,0,0
2,19,19,0,0.0,7.263158,1,1,0.103448,0,0,0,20,8.0224,0,1
3,10,10,0,0.0,6.900000,1,1,0.107143,0,0,0,12,37.7600,1,0
4,9,9,0,0.0,5.777778,1,1,0.052632,0,0,2,8,36.7360,1,1


In [271]:
X_surf_feats_tr = scipy.sparse.csr_matrix(train_data_feats.to_sparse())

In [274]:
X_feats_tr = hstack((X_title_count_tr, X_surf_feats_tr))

In [275]:
X_feats_tr

<36283x111 sparse matrix of type '<class 'numpy.float64'>'
	with 354220 stored elements in COOrdinate format>

In [197]:
'Suri Square' in brands_df

False

In [198]:
print(X.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [199]:
from sklearn.metrics import accuracy_score


In [200]:
np.array(validation_clarity_labels['clarity'])


array([0.34139, 0.22758, 0.36162, ..., 0.39207, 0.48232, 0.27523])

In [201]:
validation_data.head()

,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type
0,my,AP564ELASSTWANMY,Apple MacBook Pro MGXC2ZP/A 16GB i7 15.4-inch ...,Computers & Laptops,Laptops,Macbooks,OS X Lion<br> Intel Core i7<br> 15-inch Retina...,12550.0,local
1,my,BR924HBAA5B3TLANMY,BRAND'S® American Ginseng Triple Pack (3x 6's)...,Health & Beauty,Food Supplements,Well Being,<ul> <li>Traditionally used to calm the mind a...,105.0,local
2,my,CA673ELAA5UG3XANMY,Canon EOS M10 Mirrorless Digital Camera 18MP w...,Cameras,Mirrorless,NaN,<div> <ul> <li>18.0MP APS-C CMOS Sensor</li> <...,1588.0,local
3,my,DE759ELAA7QM1XANMY,"Dell LED Monitor 23"" (E2316H)",Computers & Laptops,Computer Accessories,Monitors,"<div class=""prod_content""> <div class=""prod_de...",565.0,local
4,my,ES802OTAABHAY8ANMY,Esprit Tallac Brave Nubuck Sand ES107601001 Be...,Watches Sunglasses Jewellery,Watches,Men,<ul> <li>stainless steel case</li> <li>mineral...,279.0,local


In [202]:
X_train

<36283x96 sparse matrix of type '<class 'numpy.int64'>'
	with 38559 stored elements in Compressed Sparse Row format>

In [205]:
X = X_train.copy().astype('float32')

In [177]:
y_pred


array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [178]:
from sklearn.metrics import accuracy_score

print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))


LightGBM Model accuracy score: 0.9429


In [206]:
X.shape


(36283, 96)

In [276]:
kf = StratifiedKFold(n_splits=10, shuffle=True)
lgb_model = lgb.LGBMClassifier()
YStrat = train_clarity_labels['clarity'].values
step = 1
for train_index, test_index in kf.split(X_feats_tr,YStrat): 
    train_X = X[train_index]
    train_Y = train_clarity_labels['clarity'][train_index]
    test_X = X[test_index]
    test_Y = train_clarity_labels['clarity'][test_index]
    lgb_model.fit(train_X, train_Y)
    predictions = lgb_model.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    print("LGBM model accuracy score for K fold {} = {}".format(step, accuracy) )
    step += 1

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGBM model accuracy score for K fold 1 = 0.9429594929732709


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGBM model accuracy score for K fold 2 = 0.9429594929732709


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGBM model accuracy score for K fold 8 = 0.9434950385887542


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGBM model accuracy score for K fold 9 = 0.9434794596084919
LGBM model accuracy score for K fold 10 = 0.9434794596084919


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [277]:
kf = StratifiedKFold(n_splits=10, shuffle=True)
naive_bayes_model = MultinomialNB()
YStrat = train_clarity_labels['clarity'].values
step = 1
for train_index, test_index in kf.split(X_feats_tr,YStrat): 
    train_X = X[train_index]
    train_Y = train_clarity_labels['clarity'][train_index]
    test_X = X[test_index]
    test_Y = train_clarity_labels['clarity'][test_index]
    naive_bayes_model.fit(train_X, train_Y)
    predictions = naive_bayes_model.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    print("Naive Bayesian model accuracy score for K fold {} = {}".format(step, accuracy) )
    step += 1

Naive Bayesian model accuracy score for K fold 1 = 0.9366216588591899
Naive Bayesian model accuracy score for K fold 2 = 0.9338660788095894
Naive Bayesian model accuracy score for K fold 3 = 0.931110498759989
Naive Bayesian model accuracy score for K fold 4 = 0.9319371727748691
Naive Bayesian model accuracy score for K fold 5 = 0.93717277486911
Naive Bayesian model accuracy score for K fold 6 = 0.9346747519294377
Naive Bayesian model accuracy score for K fold 7 = 0.9355016538037486
Naive Bayesian model accuracy score for K fold 8 = 0.9368798235942668
Naive Bayesian model accuracy score for K fold 9 = 0.9352081610146127
Naive Bayesian model accuracy score for K fold 10 = 0.9324510614833196


In [ ]:
# apparently countvectrizering title with 2,6 ngrams range gives significant performace.